In [98]:
from keras.layers import Dense, Input, Lambda, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
import keras

In [33]:
import numpy as np
from keras.layers.advanced_activations import LeakyReLU
from PIL import Image
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import os
import glob
from scipy.misc import imread,imresize
from keras.applications.vgg16 import VGG16
from skimage import color
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
from keras.optimizers import Adam





In [122]:
class ImageDataset:
    def __init__(self,image_weight = 224, image_height = 224):
        self.image_weight = image_weight
        self.image_height = image_height
        self.dataset_path = './'#'../../Descargas/AVA/ava_downloader/image/'
        self.start_idx = 0
        self.end_idx = 1
        
        self.input_image = []
        self.input_hist = []
        self.output = []
        
        
        self.dataset_load()
        
    def dataset_load(self):
        raw_image_list = glob.glob(self.dataset_path + '*.jpg')
        raw_image_list = sorted(raw_image_list)[self.start_idx:self.end_idx]
        print raw_image_list
        
        for i, img_path in enumerate(raw_image_list):
            if i%100 == 0:
                print ("processed %d out of %d" %( i, len(raw_image_list) ))
            try:
                """
                image = imresize(imread(img_path), (self.image_weight,self.image_height))#/255.0-0.5
                #image = image.img_to_array(image)
                #image = np.expand_dims(image)
                #image = image.img_to_array(image, axis = 0)
                image = preprocess_input(image)
                self.input_image.append(image)
                """
                print 'done image', i
                img = image.load_img(img_path, target_size=(224, 224))
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)
                self.input_image.append(x)
                
                
                
                color_hist = lab_hist(x)
                self.input_hist.append(np.stack(np.array(color_hist)))
                
                parameters = np.ones(6)
                image_flat = x.flatten()
                output = np.concatenate([parameters , image_flat])
                self.output.append(output)
                
            except Exception as e:
                print str(e)

    
def lab_hist(image_np):
    image_np_rgb = image_np/255.0 # 0~1 rgb
    image_np_lab = color.rgb2lab(image_np_rgb)

    num_bin_L = 10
    num_bin_a = 10
    num_bin_b = 10

    L_max = 100
    L_min = 0
    a_max = 60
    a_min = -60
    b_max = 60
    b_min = -60
    image_np_lab = image_np_lab.reshape([224*224,3])
    H, edges = np.histogramdd(image_np_lab, bins=(num_bin_L, num_bin_a, num_bin_b), \
            range=((L_min, L_max), (a_min, a_max), (b_min, b_max)))
    return H.reshape(1000)/1000.0
        
mydata = ImageDataset();
        

processed 0 out of 1
cannot identify image file './imagen.jpg'


In [112]:
class NeuralNetwork:
    def __init__(self, image_weight = 224, image_height = 224):
        self.image_size = (image_weight, image_height,3)
        self.hist_size = 1000
        self.batch_size = 4
        
        optimizer = Adam(0.0002, 0.2)
        
        self.input_image = Input(shape = self.image_size, name='input_image')
        self.feature_NN = self.build_VGG()(self.input_image)
               
        
        self.input_hist = Input(shape = (self.hist_size,), name='input_hist')
        x = keras.layers.concatenate([self.feature_NN, self.input_hist])
        self.parameters_NN = self.build_NN()(x)
        
        z = Flatten()(self.input_image)
        
        #z = Input(shape=(self.image_size))(self.input_image)
        self.output = keras.layers.concatenate([z, self.parameters_NN])

        self.combined = Model([self.input_image, self.input_hist], self.output)
        #self.combined.compile(loss = _loss_f, optimizer = optimizer)
        
        
    def build_NN(self, input_size = (5096,)):
        inputs = Input(shape = input_size)
        x = Dense(512, activation = 'relu')(inputs)
        x = Dense(64, activation = 'relu')(x)
        parameters = Dense(6, activation = 'linear')(x)
        model = Model(inputs=inputs, outputs=parameters)
        # model.summary
        return model
        
    def build_VGG(self, istrainable = False):
        model = VGG16(weights = 'imagenet')
        layer_name = 'fc2'
        model2 = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
        model2.trainable = istrainable
        return model2
        
class DeepFeatureNetwork:
    def __init__(self, input_size, model_func, model_path):
        self.input_tensor = tf.placeholder(tf.float32, shape=input_size)
        self.feature, self.weights = model_func(self.input_tensor, model_path)
        
    def init_model(self):
        tf.initialize_variables(self.weights).run()

    def get_feature(self, in_data):
        return sess.run(self.feature, feed_dict={self.input_tensor: in_data+0.5})
    
myNN = NeuralNetwork()

        

In [113]:
myNN.combined.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_47 (Model)                (None, 4096)         134260544   input_image[0][0]                
__________________________________________________________________________________________________
input_hist (InputLayer)         (None, 1000)         0                                            
__________________________________________________________________________________________________
concatenate_32 (Concatenate)    (None, 5096)         0           model_47[1][0]                   
                                                                 input_hist[0][0]                 
__________

In [101]:
#myNN = NeuralNetwork(224,224)

In [106]:
image_weight = 224
image_height = 224

image_size = (image_weight, image_height,3)
hist_size = 1000
batch_size = 4

optimizer = Adam(0.0002, 0.2)

input_image = Input(shape = image_size, name = 'input_image')


model_build_VGG = VGG16(weights = 'imagenet')
layer_name = 'fc2'
model_build_VGG_ = Model(inputs=model_build_VGG.input, outputs=model_build_VGG.get_layer(layer_name).output)
model_build_VGG_.trainable = False

feature_NN = model_build_VGG_(input_image)



#input_hist = Input(shape = (hist_size,), name='input_hist')
#x_ = keras.layers.concatenate([feature_NN, input_hist])




#input_hist = Input(shape = hist_size, name='input_hist')
#x = keras.layers.concatenate([feature_NN, input_hist])
#parameter_NN = build_NN()(x)

#z = Input(shape = (image_size))(input_image)
#output = keras.layer.concatenate([z, parameters_NN])

#combined = Model([input_image, input_hist], output)

In [107]:
myf = Flatten()(input_image)

x_ = keras.layers.concatenate([feature_NN, input_hist])
inputs = Input(shape = (5096,))
x = Dense(250, activation = 'relu')(inputs)
x = Dense(100, activation = 'relu')(x)
parameters = Dense(6, activation = 'linear')(x)
model2 = Model(inputs=inputs, outputs=parameters)

parameter_NN = model2(x_)
#z = Input(shape = image_size)
#z = z(input_image)
output = keras.layers.concatenate([myf, parameter_NN])

combined = Model([input_image, input_hist], output)

In [66]:
mymodel = Model(inputs = [input_hist, input_image], outputs = x)

In [76]:
parameter_NN.summary()
model2.summary()

AttributeError: 'Tensor' object has no attribute 'summary'

In [55]:
path=u'./imagen.jpg'
myimage = imread(name = path)

IOError: cannot identify image file u'./imagen.jpg'

In [54]:
from PIL import Image
image=Image.open(u'home/emanuelsamir/UTEC/6to_ciclo/Digital_Image_Processing/project/Park_code/DISTORT-AND-RECOVER-CVPR18/DIP_project_Testing'
)

IOError: [Errno 21] Is a directory: u'/home/emanuelsamir/UTEC/6to_ciclo/Digital_Image_Processing/project/Park_code/DISTORT-AND-RECOVER-CVPR18/DIP_project_Testing'

In [59]:
ls

Building.ipynb  Fig0458(a)(blurry_moon).tif  imagen.jpg  imagen.tiff


In [80]:
input_image = Input(shape = image_size, name = 'input_image')

In [92]:
myf

AttributeError: 'Tensor' object has no attribute 'layer'

In [100]:
from keras.utils import plot_model
plot_model(combined)